In [1]:
from sklearn.metrics import  accuracy_score
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd

In [2]:
train_size = 0.6
test_size = 0.4

In [3]:
data = load_iris()

In [4]:
dataset = data['data']
target = data['target'].reshape(-1,1)

In [5]:
final_data = np.hstack((dataset,target))
np.random.shuffle(final_data)

In [6]:
pd_data = pd.DataFrame(final_data)
pd_data.columns= ['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)','target']
pd_data = pd_data[(pd_data['target']==0) | (pd_data['target']==1)]
pd_data.tail(2)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
145,5.0,3.5,1.3,0.3,0.0
146,6.1,2.8,4.7,1.2,1.0


In [7]:
final_data = pd_data.values

train_data = final_data[:int(len(final_data)*train_size)]
test_data = final_data[:int(len(final_data)*test_size)]

train_data.shape,test_data.shape

((60, 5), (40, 5))

In [8]:
X_train = train_data[:,:-1]
y_train = train_data[:,-1:]


X_test = test_data[:,:-1]
y_test = test_data[:,-1:]


X_train.shape,y_train.shape,X_test.shape,y_test.shape

((60, 4), (60, 1), (40, 4), (40, 1))

In [9]:
def gradient(y,y_pred,x):
    return np.dot(x.T,(y-y_pred))

In [10]:
def new_grad(x):
    return sigmoid(x)(1-sigmoid(x))

In [11]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [12]:
def cost(y,y_pred):
    cost = -np.mean((y*np.log(y_pred))+((1-y)*(np.log(1-y_pred))))
    return cost

In [13]:
dummy_train = np.hstack((np.ones((len(train_data),1)),X_train))
weight = np.zeros((len(dummy_train[0]),1))
lr = 0.001
epoch = 1000


for i in range(epoch):
    y_pred = sigmoid(np.dot(dummy_train,weight))
    co = cost(y_train,y_pred)
    gra = gradient(y_train,y_pred,dummy_train)
    weight += lr*gra
    
    
    if i%10 == 0:
        print('Epoch : ',i,' Loss:',co)

Epoch :  0  Loss: 0.6931471805599454
Epoch :  10  Loss: 0.4664291766402609
Epoch :  20  Loss: 0.35583248173654897
Epoch :  30  Loss: 0.28388961752230385
Epoch :  40  Loss: 0.234643686110827
Epoch :  50  Loss: 0.1993116173297633
Epoch :  60  Loss: 0.1729357182569546
Epoch :  70  Loss: 0.1525904029979892
Epoch :  80  Loss: 0.1364677560678673
Epoch :  90  Loss: 0.12340218344669138
Epoch :  100  Loss: 0.1126135738988026
Epoch :  110  Loss: 0.10356252701739771
Epoch :  120  Loss: 0.0958653693310874
Epoch :  130  Loss: 0.08924237760294493
Epoch :  140  Loss: 0.08348516090185978
Epoch :  150  Loss: 0.07843548858218807
Epoch :  160  Loss: 0.07397117581050786
Epoch :  170  Loss: 0.069996446565363
Epoch :  180  Loss: 0.06643521117363786
Epoch :  190  Loss: 0.06322628553865252
Epoch :  200  Loss: 0.060319931390357594
Epoch :  210  Loss: 0.057675312585096665
Epoch :  220  Loss: 0.05525859775614366
Epoch :  230  Loss: 0.05304152631224878
Epoch :  240  Loss: 0.05100031146064823
Epoch :  250  Loss: 0

In [14]:
weight

array([[-0.30373647],
       [-0.47417418],
       [-1.76281071],
       [ 2.74306895],
       [ 1.23296972]])

In [15]:
def predict(x,w):
    sig = sigmoid(np.dot(x,w[1:])+w[0])
    sig[sig > 0.5] = 1
    sig[sig < 0.5] = 0
    return sig

In [16]:
accuracy_score(predict(X_test,weight),y_test)

1.0

In [31]:
class logisticregression():
    def __init__(self,train_data,train_labels,lr=0.01,batch_size=None,epoch=10):
        dummy_once = np.ones((len(train_data),1))
        self.train_data = np.hstack((dummy_once,train_data))
        self.train_labels = train_labels
        
        self.params = np.zeros((len(self.train_data[0]),1))
        
        self.lr = lr
        self.epoch = epoch
        self.batch_size = batch_size
        
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def cost(self,y,y_pred):
        return -np.mean(y*np.log(y_pred)+(1-y)*np.log(1-y_pred))
    
    def gradient(self,y,y_pred,x):
        return np.dot(x.T,(y_pred-y))
    
    def train(self):
        for i in range(self.epoch):
            y_pred = self.sigmoid(np.dot(self.train_data,self.params))
            loss = self.cost(self.train_labels,y_pred)
            
            gra = self.gradient(self.train_labels,y_pred,self.train_data)
            
            self.params -= self.lr*gra
            
            if i%10 == 0 or i == self.epoch-1:
                print('Epoch : {}  Loss: {}'.format(i,loss))
    def predict(self,test_data):
        result = sigmoid(np.dot(test_data,self.params[1:])+self.params[0])
        result[result >= 0.5 ] = 1
        result[result < 0.5 ] = 0
        return result
    
    def evaluate(self,test_data,labels):
        accuracy = accuracy_score(self.predict(test_data),labels)
        return accuracy

In [32]:
logistic = logisticregression(X_train,y_train,epoch=10)

In [33]:
logistic.train()

Epoch : 0  Loss: 0.6931471805599454
Epoch : 9  Loss: 0.035442194456427614


In [34]:
logistic.evaluate(X_test,y_test)

1.0